In [9]:
from transformers import MBartForConditionalGeneration, AutoTokenizer

from huggingface_hub import notebook_login
from transformers import MBart50Tokenizer
import tqdm
import torch
import evaluate
import pandas as pd

import os

In [10]:
val = pd.read_json('/mnt/disk/yrajcoomar/kreol-benchmark/data/lang_data/en-cr/en-cr_dev.jsonl',lines=True)
val_inputs = list(val['input'])
val_labels = list(val['target'])

In [11]:
bleu = evaluate.load("bleu")
chrf = evaluate.load("chrf")

In [12]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device='cpu'

In [13]:
weights = os.listdir('/mnt/disk/yrajcoomar/kreol-benchmark/checkpoint')

In [15]:
for ckpt in weights:
    checkpoint = os.path.join('/mnt/disk/yrajcoomar/kreol-benchmark/checkpoint',ckpt)
    tokenizer = AutoTokenizer.from_pretrained(checkpoint)
    model = MBartForConditionalGeneration.from_pretrained(checkpoint)
    model = model.to(device)
    input_tokens = tokenizer(val_inputs,max_length=128, truncation=True, padding="max_length",return_tensors='pt').to(device)
    output_tokens = model.generate(**input_tokens)
    output = tokenizer.batch_decode(output_tokens, skip_special_tokens=True)
    print(checkpoint)
    print(chrf.compute(predictions=output,references=val_labels))
    print(bleu.compute(predictions=output,references=val_labels))
    print('----------------')

/mnt/disk/yrajcoomar/kreol-benchmark/checkpoint/checkpoint-140000
{'score': 44.84547543879345, 'char_order': 6, 'word_order': 0, 'beta': 2}
{'bleu': 0.2206957217203022, 'precisions': [0.5968494749124854, 0.3073110285006196, 0.17582562747688243, 0.10352142554094187], 'brevity_penalty': 0.9181318826103453, 'length_ratio': 0.9213072457536013, 'translation_length': 8570, 'reference_length': 9302}
----------------
/mnt/disk/yrajcoomar/kreol-benchmark/checkpoint/checkpoint-20000
{'score': 43.9885408439185, 'char_order': 6, 'word_order': 0, 'beta': 2}
{'bleu': 0.21161918530428125, 'precisions': [0.5817245817245817, 0.29476823660991675, 0.16960381608586192, 0.0981838819523269], 'brevity_penalty': 0.9154540673427985, 'length_ratio': 0.9188346592130725, 'translation_length': 8547, 'reference_length': 9302}
----------------
/mnt/disk/yrajcoomar/kreol-benchmark/checkpoint/checkpoint-60000
{'score': 43.022916426277334, 'char_order': 6, 'word_order': 0, 'beta': 2}
{'bleu': 0.20311427379314842, 'prec